In [24]:
import sqlite3
import pandas as pd
from sklearn.metrics.pairwise import cosine_similarity

### 필요한 데이터 추출

In [60]:
conn = sqlite3.connect('cp2_data.db')
cur = conn.cursor()

query = cur.execute("""SELECT user_id, product_id, category_code, event_type, main_category, brand, price
FROM
	(SELECT *, substr(category_code, 1, pos-1) AS main_category
	FROM
	  (SELECT *,
	          instr(category_code ,'.') AS pos
	   FROM "2019-Oct"))
WHERE category_code != '' AND brand != '' LIMIT 100000 """)

cols = [column[0] for column in query.description]

df = pd.DataFrame.from_records(data=query.fetchall(), columns=cols)

conn.close()

In [61]:
df

,user_id,product_id,category_code,event_type,main_category,brand,price
0,554748717,3900821,appliances.environment.water_heater,view,appliances,aqua,33.20
1,550050854,1307067,computers.notebook,view,computers,lenovo,251.74
2,535871217,1004237,electronics.smartphone,view,electronics,apple,1081.98
3,512742880,1480613,computers.desktop,view,computers,pulser,908.62
4,520571932,28719074,apparel.shoes.keds,view,apparel,baden,102.71
...,...,...,...,...,...,...,...
99995,533787062,5100239,electronics.clocks,view,electronics,samsung,257.38
99996,512524227,4400363,appliances.kitchen.coffee_machine,view,appliances,polaris,25.71
99997,513606959,1004426,electronics.smartphone,view,electronics,samsung,217.04
99998,536181285,28401078,accessories.bag,view,accessories,respect,66.67


### view는 1점, cart는 2점, purchase는 3점 부여

In [16]:
df.loc[df["event_type"] == "view", "rating"] = 1
df.loc[df["event_type"] == "cart", "rating"] = 2
df.loc[df["event_type"] == "purchase", "rating"] = 3
df

,user_id,product_id,category_code,event_type,main_category,rating
0,554748717,3900821,appliances.environment.water_heater,view,appliances,1.0
1,550050854,1307067,computers.notebook,view,computers,1.0
2,535871217,1004237,electronics.smartphone,view,electronics,1.0
3,512742880,1480613,computers.desktop,view,computers,1.0
4,520571932,28719074,apparel.shoes.keds,view,apparel,1.0
...,...,...,...,...,...,...
99995,533787062,5100239,electronics.clocks,view,electronics,1.0
99996,512524227,4400363,appliances.kitchen.coffee_machine,view,appliances,1.0
99997,513606959,1004426,electronics.smartphone,view,electronics,1.0
99998,536181285,28401078,accessories.bag,view,accessories,1.0


In [17]:
df['user_id'].value_counts()

529826736    122
522799138    117
513062867    101
513093544     93
555462920     91
            ... 
528106562      1
555485222      1
517493111      1
527991525      1
536181285      1
Name: user_id, Length: 22209, dtype: int64

### user_id와 product_id 별로 합친 점수

In [18]:
df = df.groupby(['user_id', 'product_id'], as_index=False).sum()
df

,user_id,product_id,rating
0,306441847,2501614,2.0
1,351866718,16900161,1.0
2,370076704,4700546,1.0
3,386070015,1004133,1.0
4,386070015,1004739,4.0
...,...,...,...
62439,555497326,7005315,1.0
62440,555497368,2501534,1.0
62441,555497368,4500301,1.0
62442,555497404,13800004,1.0


### 제품 별 유사도를 측정하기 위해 pivot table 구성

In [20]:
product_user = df.pivot_table('rating', index = 'product_id', columns='user_id')
product_user

user_id,306441847,351866718,370076704,386070015,409686912,414175028,414824833,418226573,421257570,424357361,...,555497146,555497163,555497183,555497203,555497301,555497304,555497326,555497368,555497404,555497481
product_id,,,,,,,,,,,,,,,,,,,,,
1001588,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1002062,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1002098,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1002099,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1002100,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
9900426,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
9900444,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
9900447,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [21]:
product_user.fillna(0, inplace = True)
product_user

user_id,306441847,351866718,370076704,386070015,409686912,414175028,414824833,418226573,421257570,424357361,...,555497146,555497163,555497183,555497203,555497301,555497304,555497326,555497368,555497404,555497481
product_id,,,,,,,,,,,,,,,,,,,,,
1001588,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1002062,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1002098,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1002099,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1002100,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
9900426,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
9900444,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
9900447,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


### 코사인 유사도

In [25]:
item_based_collabor = cosine_similarity(product_user)
item_based_collabor

array([[1.        , 0.        , 0.        , ..., 0.        , 0.        ,
        0.        ],
       [0.        , 1.        , 0.        , ..., 0.        , 0.        ,
        0.        ],
       [0.        , 0.        , 1.        , ..., 0.        , 0.        ,
        0.        ],
       ...,
       [0.        , 0.        , 0.        , ..., 1.        , 0.        ,
        0.        ],
       [0.        , 0.        , 0.        , ..., 0.        , 1.        ,
        0.26967994],
       [0.        , 0.        , 0.        , ..., 0.        , 0.26967994,
        1.        ]])

In [26]:
item_based_collabor = pd.DataFrame(data = item_based_collabor, index = product_user.index, columns = product_user.index)
item_based_collabor

product_id,1001588,1002062,1002098,1002099,1002100,1002101,1002102,1002225,1002396,1002398,...,9900221,9900249,9900399,9900409,9900420,9900426,9900444,9900447,9900448,9900453
product_id,,,,,,,,,,,,,,,,,,,,,
1001588,1.0,0.0,0.000000,0.000000,0.000000,0.000000,0.000000,0.0,0.000000,0.0,...,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,0.0,0.000000,0.000000
1002062,0.0,1.0,0.000000,0.000000,0.000000,0.000000,0.000000,0.0,0.000000,0.0,...,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,0.0,0.000000,0.000000
1002098,0.0,0.0,1.000000,0.105063,0.061256,0.087163,0.099449,0.0,0.000000,0.0,...,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,0.0,0.000000,0.000000
1002099,0.0,0.0,0.105063,1.000000,0.135956,0.148811,0.215064,0.0,0.200895,0.0,...,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,0.0,0.000000,0.000000
1002100,0.0,0.0,0.061256,0.135956,1.000000,0.069411,0.131991,0.0,0.000000,0.0,...,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,0.0,0.000000,0.000000
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
9900426,0.0,0.0,0.000000,0.000000,0.000000,0.000000,0.000000,0.0,0.000000,0.0,...,0.0,0.0,0.0,0.0,0.0,1.000000,0.0,0.0,0.447214,0.603023
9900444,0.0,0.0,0.000000,0.000000,0.000000,0.000000,0.000000,0.0,0.000000,0.0,...,0.0,0.0,0.0,0.0,0.0,0.000000,1.0,0.0,0.000000,0.000000
9900447,0.0,0.0,0.000000,0.000000,0.000000,0.000000,0.000000,0.0,0.000000,0.0,...,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,1.0,0.000000,0.000000


### 입력한 product_id에 해당하는 제품과 비슷한 10개의 제품 출력(입력된 제품 포함)

In [68]:
def get_item_based_collabor(product_id):
    return item_based_collabor[product_id].sort_values(ascending=False)[:10]

In [69]:
get_item_based_collabor('1001588')

product_id
1001588    1.000000
1004143    0.226294
1004855    0.209913
1004415    0.202953
1004144    0.167716
1003114    0.157895
1004205    0.155907
1002482    0.145095
1004814    0.139942
1004024    0.127257
Name: 1001588, dtype: float64

### 입력한 product_id에 해당하는 제품과 비슷한 10개의 제품 정보 출력(입력된 제품 포함)

In [73]:
def get_item_based_collabor(product_id):
    recommend = item_based_collabor[product_id].sort_values(ascending=False)[:10].index
    recommend_list = []
    for i in recommend:
        recommend_list.append({'product id': i, 'category code': df[df['product_id'] == i].reset_index(drop=True).iloc[0]['category_code'], 'brand': df[df['product_id'] == i].reset_index(drop=True).iloc[0]['brand'], 'price': df[df['product_id'] == i].reset_index(drop=True).iloc[0]['price']})
    return recommend_list

In [74]:
get_item_based_collabor('1001588')

[{'product id': '1001588',
  'category code': 'electronics.smartphone',
  'brand': 'meizu',
  'price': '128.31'},
 {'product id': '1004143',
  'category code': 'electronics.smartphone',
  'brand': 'xiaomi',
  'price': '141.32'},
 {'product id': '1004855',
  'category code': 'electronics.smartphone',
  'brand': 'xiaomi',
  'price': '126.03'},
 {'product id': '1004415',
  'category code': 'electronics.smartphone',
  'brand': 'bq',
  'price': '42.45'},
 {'product id': '1004144',
  'category code': 'electronics.smartphone',
  'brand': 'xiaomi',
  'price': '128.38'},
 {'product id': '1003114',
  'category code': 'electronics.smartphone',
  'brand': 'sony',
  'price': '128.39'},
 {'product id': '1004205',
  'category code': 'electronics.smartphone',
  'brand': 'xiaomi',
  'price': '128.59'},
 {'product id': '1002482',
  'category code': 'electronics.smartphone',
  'brand': 'meizu',
  'price': '128.31'},
 {'product id': '1004814',
  'category code': 'electronics.smartphone',
  'brand': 'honor